# Mecab

- 自分で単語を追加することもできる。

- install手順
- [MacにMecabを入れて形態素解析をする](https://yoshikiito.net/blog/archives/mac-mecab/)
```
brew install mecab
brew install mecab-ipadic
pip install mecab-python3
```

In [1]:
import MeCab

In [2]:
# MeCabオブジェクトの生成 --- (*1)
tagger = MeCab.Tagger()
# 形態素解析 --- (*2)
result = tagger.parse("メイが恋ダンスを踊っている。")
print(result)

メイ	名詞,一般,*,*,*,*,*
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
恋	名詞,一般,*,*,*,*,恋,コイ,コイ
ダンス	名詞,サ変接続,*,*,*,*,ダンス,ダンス,ダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
。	記号,句点,*,*,*,*,。,。,。
EOS



# mecab-ipadic-neologd

- インストール手順
- [【Ubuntu】MeCabとNEologdをインストールしてPythonで形態素解析する](https://engineeeer.com/mecab-neologd-python/)
```
git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
cd mecab-ipadic-neologd
./bin/install-mecab-ipadic-neologd -n
echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
```

```
/opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd
```

In [3]:
# mecab-ipadic-NEologd辞書を指定して、MeCabオブジェクトの生成 --- (*1)
tagger = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd")
# 形態素解析
result = tagger.parse("メイが恋ダンスを踊っている。")
print(result)

メイ	名詞,固有名詞,人名,一般,*,*,M.A.Y,メイ,メイ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
恋ダンス	名詞,固有名詞,一般,*,*,*,恋ダンス,コイダンス,コイダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
。	記号,句点,*,*,*,*,。,。,。
EOS



# ストップワードの除去

- 助詞や助動詞などの利用頻度の高いワードを除去する。
- 以下は品詞情報を用いて除去する。

In [4]:
tagger = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd")
tagger.parse("") 
# 形態素解析結果をリストで取得 --- (*1)
node = tagger.parseToNode("メイが恋ダンスを踊っている。")

result = []
while node is not None:
    # 品詞情報取得 --- (*2)
    hinshi = node.feature.split(",")[0]
    if  hinshi in ["名詞"]:
        # 表層形の取得 --- (*3)
        result.append(node.surface)
    elif hinshi in ["動詞", "形容詞"]:
        # 形態素情報から原形情報を取得 --- (*4)
        # 品詞が動詞や形容詞の場合、送りがななどに違いが出るため、ストップワードの除去とは関係してないが、原型を取得することで正規化。
        result.append(node.feature.split(",")[6])
    node = node.next
    
print(result)


['メイ', '恋ダンス', '踊る', 'いる']


# 単語ベクトル

- 単語の意味を計算。
- 単語の類似度を計算。

# Word2Vec

- 単語をベクトル化できる。
- 「単語はその周りにある単語と関係がある」ということを学習する。
- 反対の意味を持ちながら、同じように使われてしまう単語が近いベクトルを持つという弱点がある。
- 「Skip-gram」と「CBOW」という２つのアルゴリズムが存在する。
- Skip-gram
    - 精度が高く速度が遅い。
- CBOW
    - 精度が比較的低く、速度が速い。
    
- 何が自分の目的に合うかを考慮して「学習させる文章」を選ぶ必要がある。

```

```


# Doc2Vec
- 任意の長さの文章をベクトル化できる。
- Word2Vecよりも複雑な計算をするので、より多くのメモリが必要になる。

今回のプログラムを応用することで、スパム判定、問い合わせの分類などもできる。

In [5]:
import zipfile
import os.path
import urllib.request as req
import MeCab
from gensim import models
from gensim.models.doc2vec import TaggedDocument

In [6]:
#Mecabの初期化
mecab = MeCab.Tagger()
mecab.parse("")

#学習対象とする青空文庫の作品リスト --- (*1)
# ZIPファイルをダウンロード
# 4人の作家のそれぞれ5作品ずつダウンロードするための情報を用意。
list = [
    {"auther":{
        "name":"宮澤 賢治",
        "url":"https://www.aozora.gr.jp/cards/000081/files/"}, 
     "books":[
        {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
        {"name":"注文の多い料理店","zipname":"1927_ruby_17835.zip"},
        {"name":"セロ弾きのゴーシュ","zipname":"470_ruby_3987.zip"},
        {"name":"やまなし","zipname":"46605_ruby_29758.zip"},
        {"name":"どんぐりと山猫","zipname":"43752_ruby_17595.zip"},
    ]},
    {"auther":{
        "name":"芥川 竜之介",
        "url":"https://www.aozora.gr.jp/cards/000879/files/"}, 
     "books":[
        {"name":"羅生門","zipname":"127_ruby_150.zip"},
        {"name":"鼻","zipname":"42_ruby_154.zip"},
        {"name":"河童","zipname":"69_ruby_1321.zip"},
        {"name":"歯車","zipname":"42377_ruby_34744.zip"},
        {"name":"老年","zipname":"131_ruby_241.zip"},
    ]},
    {"auther":{
        "name":"ポー エドガー・アラン",
        "url":"https://www.aozora.gr.jp/cards/000094/files/"}, 
     "books":[
        {"name":"ウィリアム・ウィルスン","zipname":"2523_ruby_19896.zip"},
        {"name":"落穴と振子","zipname":"1871_ruby_17551.zip"},
        {"name":"黒猫","zipname":"530_ruby_20931.zip"},
        {"name":"群集の人","zipname":"56535_ruby_69925.zip"},
        {"name":"沈黙","zipname":"56537_ruby_70425.zip"},
    ]},
    {"auther":{
        "name":"紫式部",
        "url":"https://www.aozora.gr.jp/cards/000052/files/"}, 
     "books":[
        {"name":"源氏物語 01 桐壺","zipname":"5016_ruby_9746.zip"},
        {"name":"源氏物語 02 帚木","zipname":"5017_ruby_9752.zip"},
        {"name":"源氏物語 03 空蝉","zipname":"5018_ruby_9754.zip"},
        {"name":"源氏物語 04 夕顔","zipname":"5019_ruby_9761.zip"},
        {"name":"源氏物語 05 若紫","zipname":"5020_ruby_11253.zip"},
    ]},
]

#作品リストを取得してループ処理に渡す --- (*2)
# 「著者と作品」という単位で返す関数を定義。
def book_list():
    for novelist in list:
        auther = novelist["auther"]
        for book in novelist["books"]:
            yield auther, book
        
#Zipファイルを開き、中の文書を取得する --- (*3)
# テキストファイルを文字列にして返す。
def read_book(auther, book):
    zipname = book["zipname"]
    #Zipファイルが無ければ取得する
    if not os.path.exists(zipname):
        req.urlretrieve(auther["url"] + zipname, zipname)
    zipname = book["zipname"]
    #Zipファイルを開く
    with zipfile.ZipFile(zipname,"r") as zf:
        #Zipファイルに含まれるファイルを開く。
        for filename in zf.namelist():
            # テキストファイル以外は処理をスキップ
            if os.path.splitext(filename)[1] != ".txt":
                continue
            with zf.open(filename,"r") as f: 
                #今回読むファイルはShift-JISなので指定してデコードする
                return f.read().decode("shift-jis")

#引数のテキストを分かち書きして配列にする ---(*4)
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

#作品リストをDoc2Vecが読めるTaggedDocument形式にし、配列に追加する --- (*5)
documents = []
#作品リストをループで回す
for auther, book in book_list():
    #作品の文字列を取得
    words = read_book(auther, book)
    #作品の文字列を分かち書きに
    wakati_words = split_words(words)
    #TaggedDocumentの作成　文書=分かち書きにした作品　タグ=作者:作品名
    document = TaggedDocument(
        wakati_words, [auther["name"] + ":" + book["name"]])
    documents.append(document)
    
#TaggedDocumentの配列を使ってDoc2Vecの学習モデルを作成 --- (*6)
'''
dm : Doc2Vecで使うアルゴリズムを選択する。1=dmpw, 0=DBOW
size : ベクトルの次元を設定。Doc2Vecでは基本的に300が良いとされている。
window : 学習する単語の前後数。DBOWでは15が良いとされている。
min_count : 最低何回出てきた文字列を対象とするかの設定。
（今回は作家ごとに独特な言い回しなどがあると考えられるため、一回でも出てきたとき文字列を対象にする。)
'''
model = models.Doc2Vec(
    documents, dm=0, vector_size=300, window=15, min_count=1)

#Doc2Vecの学習モデルを保存
model.save('aozora.model')

print("モデル作成完了")

モデル作成完了


In [7]:
#Mecabの初期化
mecab = MeCab.Tagger()
mecab.parse("")

#保存したDoc2Vec学習モデルを読み込み --- (*7)
model = models.Doc2Vec.load('aozora.model')

#分類用のZipファイルを開き、中の文書を取得する --- (*8)
def read_book(url, zipname):
    if not os.path.exists(zipname):
        req.urlretrieve(url, zipname)

    with zipfile.ZipFile(zipname,"r") as zf:
        for filename in zf.namelist():
            with zf.open(filename,"r") as f:
                return f.read().decode("shift-jis")

#引数のテキストを分かち書きして配列にする
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

#引数のタイトル、URLの作品を分類する --- (*9)
# もっとも類似している３作品を表示している。
def similar(title, url):
    zipname = url.split("/")[-1]
        
    words = read_book(url, zipname)
    wakati_words = split_words(words)
    vector = model.infer_vector(wakati_words)
    print("--- 「" + title + '」 と似た作品は? ---')
    print(model.docvecs.most_similar([vector],topn=3))
    print("")

#各作家の作品を１つずつ分類 --- (*10)
similar("宮沢 賢治:よだかの星",
        "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

similar("芥川 龍之介:犬と笛",
        "https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")

similar("ポー エドガー・アラン:マリー・ロジェエの怪事件",
        "https://www.aozora.gr.jp/cards/000094/files/4261_ruby_54182.zip")

similar("紫式部:源氏物語 06 末摘花",
        "https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip")

--- 「宮沢 賢治:よだかの星」 と似た作品は? ---
[('宮澤 賢治:セロ弾きのゴーシュ', 0.9809406995773315), ('宮澤 賢治:注文の多い料理店', 0.9775956869125366), ('宮澤 賢治:どんぐりと山猫', 0.9771565198898315)]

--- 「芥川 龍之介:犬と笛」 と似た作品は? ---
[('芥川 竜之介:老年', 0.9604540467262268), ('芥川 竜之介:鼻', 0.9332523345947266), ('宮澤 賢治:やまなし', 0.9224491119384766)]

--- 「ポー エドガー・アラン:マリー・ロジェエの怪事件」 と似た作品は? ---
[('ポー エドガー・アラン:黒猫', 0.8875123858451843), ('ポー エドガー・アラン:ウィリアム・ウィルスン', 0.8737896084785461), ('ポー エドガー・アラン:落穴と振子', 0.8351300358772278)]

--- 「紫式部:源氏物語 06 末摘花」 と似た作品は? ---
[('紫式部:源氏物語 01 桐壺', 0.9313539862632751), ('紫式部:源氏物語 02 帚木', 0.918245255947113), ('紫式部:源氏物語 05 若紫', 0.9100158214569092)]



# マルコフ連鎖

- 未来の状態が現在の状態のみで決まる（過去の状態とは無関係である。）という性質を持つ確率過程のこと。
- これを利用することで、既存の文章を利用し、自動で文章を生成することができる。

#### マルコフ連鎖を使った自動作文の３ステップ
1. 入力された文章を単語に分割（形態素解析）
2. 辞書を作成。
    - 文章を構成する各単語について前後の結びつきを登録。
3. 始点となる単語と辞書を使用して作文。
    - 辞書に登録されている同じ組み合わせを持つ単語をランダムに選択して繋げる。
    - 「魚」→「は」→「好き」というように１つの単語から次の単語を推測する。

In [8]:
# dict_file = "markov_dict.json"
# dic = {}

# # 辞書への登録 --- (*1)
# # 形態素解析した結果の単語リストの内容をマルコフ連鎖の辞書へ登録し、外部ファイルとして保存している。
# # 外部ファイルに保存することで、辞書の内容を蓄積して、自動作文器を育てることができる。
# def regist_dic(wordlist):
#     global dic
#     w1 = ""
#     w2 = ""
    
#     # 要素が3未満の場合は、何もしない
#     if len(wordlist) < 3 : return
    
#     for w in wordlist :
#         word = w[0]
#         if word == "" or  word == "\r\n" or word == "\n" : continue
#         # 辞書に単語を設定
#         if w1 and w2 :
#             set_dic(dic,w1, w2, word)
#         # 文末を表す語のの場合、連鎖をクリアする
#         if word == "。" or word == "?" or  word == "？" :
#             w1 = ""
#             w2 = ""
#             continue
#         # 次の前後関係を登録するために、単語をスライド
#         w1, w2 = w2, word
    
#     # 辞書を保存
#     json.dump(dic, open(dict_file,"w", encoding="utf-8"))

# # 辞書に単語を設定 --- (*2)
# # 作成した辞書と始点となる単語から、応答分を作成している。
# def set_dic(dic, w1, w2, w3):
#     # 新しい単語の場合は、新しい辞書オブジェクトを作成
#     if w1 not in dic : dic[w1] = {}
#     if w2 not in dic[w1] : dic[w1][w2] = {}
#     if w3 not in dic[w1][w2]: dic[w1][w2][w3] = 0
#     # 単語の出現数をインクリメントする
#     dic[w1][w2][w3] += 1

# # 応答文の作成 --- (*3)
# # 標準入力から文章を受け取ったのち、形態素解析を行う。
# # 応答文については、品詞が「感動詞」の場合にはそのまま返し、品詞が「名詞」「形容詞」「動詞」で辞書に登録されている場合には、その単語を始点に応答文を作成。
# def make_response(word):
#     res = []
    
#     # 「名詞」/「形容詞」/「動詞」は、文章の意図を示していることが多いと想定し、始点の単語とする。
#     w1 = word
#     res.append(w1)
#     w2 = word_choice(dic[w1])
#     res.append(w2)
#     while True:
#         # w1,w2の組み合わせから予想される、単語を選択
#         if w1 in dic and w2 in dic[w1] : w3 = word_choice(dic[w1][w2])
#         else : w3 = ""
#         res.append(w3)
#         # 文末を表す語の場合、作文を終了
#         if w3 == "。" or w3 == "?" or  w3 == "？"  or w3 == "" :  break
#         # 次の単語を選択するために、単語をスライド
#         w1, w2 = w2, w3
#     return "".join(res)
        
# def word_choice(candidate):
#     keys = candidate.keys()
#     return random.choice(list(keys))

# # メイン処理 --- (*4)

# # 辞書がすでに存在する場合は、最初に読み込む
# if os.path.exists(dict_file):
#         dic = json.load(open(dict_file,"r"))
        
# while True:
#     # 標準入力から入力を受け付け、「さようなら」が入力されるまで続ける
#     text = input("You -> ")
#     if text == "" or text == "さようなら" : 
#         print("Bot -> さようなら")
#         break

#     # 文章整形
#     if text[-1] != "。" and text[-1] != "?" and text[-1] != "？" : text +="。"
    
#     # 形態素解析
#     tagger = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd")
#     tagger.parse("") 
#     node =  tagger.parseToNode(text)
    
#     # 形態素解析の結果から、単語と品詞情報を抽出
#     wordlist = []
#     while node is not None:
#         hinshi = node.feature.split(",")[0]
#         if  hinshi not  in ["BOS/EOS"]:
#             wordlist.append([node.surface,hinshi])
#         node = node.next
    
#     # マルコフ連鎖の辞書に登録
#     regist_dic(wordlist)

#     # 応答文の作成
#     for w in wordlist:
#         word = w[0]
#         hinshi = w[1]
#         # 品詞が「感動詞」の場合は、単語をそのまま返す
#         if hinshi in [ "感動詞"] : 
#             print("Bot -> " + word)
#             break
#         # 品詞が「名詞」「形容詞」「動詞」の場合で、かつ、辞書に単語が存在する場合は、作文して返す
#         elif (hinshi in [ "名詞" ,"形容詞","動詞"]) and (word in dic):
#             print("Bot -> " + make_response(word))
#             break

# ベイジアンフィルター
- 単純ベイズ分類器を応用したもの。
- 統計的な手法でスパムを判定。

#### テキストデータの学習方法
- Bowの手法を用いた手順。
1. テキストを形態素解析して単語に分ける。
    - 「ネコに小判と言うがネコにはネコの世界がある」
    - ネコ | に | 小判 | と | 言う | が | ネコ | に | は | ネコ | の | 世界 | が | ある |
2. ストップワードを取り除く
    - ネコ | 小判 | 言う | ネコ | ネコ | 世界 | ある |
3. 単語辞書を作り、単語にIDを振る

| 単語 | ID |
| ---- | ---- |
| ネコ | 0 |
| 小判 | 1 |
| 言う | 2 |
| 世界 | 3 |
| ある | 4 |

4. ファイルごとの単語の出現頻度を調べる

| ID | 出現回数 | 出現頻度 |
| ---- | ---- | ---- |
| 0 | 3 | 0.43 |
| 1 | 1 | 0.14 |
| 2 | 1 | 0.14 |
| 3 | 1 | 0.14 |
| 4 | 1 | 0.14 |

5. 単語の出現頻度データをもとにok(0)とspam(1)に分けて学習。

- スパムテキスト → [https://github.com/kujirahand/spam-database-ja](https://github.com/kujirahand/spam-database-ja)
- 非スパムテキスト → [ldcc-20140209.tar.gz](https://www.rondhuit.com/download.html#ldcc)

In [9]:
# import glob

# # 保存ファイル名
# savefile = "ok-spam.pickle"
# # MeCabの準備 --- (*1)
# tagger = MeCab.Tagger()
# # ファイル全体で利用する変数の準備 --- (*2)
# word_dic = {"__id": 0} # 単語辞書。単語とIDを記録。word_dic["__id"]にIDを発行した単語の個数を記録し、未知語に新しいID番号を記録している。
# files = [] # 読み込んだ単語データを追加する。ファイルを読み込み、IDに変換した単語リストとラベルを追加。

# # 指定したディレクトリ内のファイル一覧を読む --- (*3)
# def read_files(dir, label):
#     # テキストファイルの一覧を得る
#     files = glob.glob(dir + '/*.txt')
#     for f in files:
#         read_file(f, label)

# # ファイルを読む --- (*4)
# # 指定されたファイルを読み込んで、文章を単語IDのリストに変換。
# def read_file(filename, label):
#     words = []
#     # ファイルの内容を読む
#     with open(filename, "rt", encoding="utf-8") as f:
#         text = f.read()
#     files.append({
#         "label": label,
#         "words": text_to_ids(text)
#     })

# # テキストを単語IDのリストに変換
# def text_to_ids(text):
#     # 形態素解析 --- (*5)
#     word_s = tagger.parse(text)
#     words = []
#     # 単語を辞書に登録 --- (*6)
#     # 単語IDに変換。
#     for line in word_s.split("\n"):
#         if line == 'EOS' or line == '': continue
#         word = line.split("\t")[0]
#         params = line.split("\t")[4].split("-")
#         hinsi = params[0] # 品詞
#         hinsi2 = params[1]  if len(params) > 1 else '' # 品詞の説明
#         org = line.split("\t")[3]  # 単語の原型
#         # 助詞・助動詞・記号・数字は捨てる --- (*7)
#         if not (hinsi in ['名詞', '動詞', '形容詞']): continue
#         if hinsi == '名詞' and hinsi2 == '数詞': continue
#         # 単語をidに変換 --- (*8)
#         id = word_to_id(org)
#         words.append(id)
#     return words

# # 単語をidに変換 --- (*9)
# # 変数wordに単語IDを追加。
# def word_to_id(word):
#     # 単語が辞書に登録されているか？
#     if not (word in word_dic):
#         # 登録されていないので新たにIDを割り振る
#         id = word_dic["__id"]
#         word_dic["__id"] += 1
#         word_dic[word] = id
#     else:
#         # 既存の単語IDを返す
#         id = word_dic[word]
#     return id

# # 単語の頻出頻度のデータを作る --- (*10)
# def make_freq_data_allfiles():
#     y = []
#     x = []
#     for f in files:
#         y.append(f['label'])
#         x.append(make_freq_data(f['words']))
#     return y, x

# def make_freq_data(words):
#     # 単語の出現回数を調べる
#     cnt = 0
#     dat = np.zeros(word_dic["__id"], 'float')
#     for w in words:
#         dat[w] += 1
#         cnt += 1
#     # 回数を出現頻度に直す --- (*11)
#     # 単語ごとに出現回数をそう出現回数でわる。
#     dat = dat / cnt
#     return dat

# # ファイルの一覧から学習用のデータベースを作る
# read_files("ok", 0)
# read_files("spam", 1)
# y, x = make_freq_data_allfiles()
# # ファイルにデータを保存
# pickle.dump([y, x, word_dic], open(savefile, 'wb'))
# print("単語頻出データ作成完了")

In [10]:
# import pickle
# from sklearn.naive_bayes import GaussianNB
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # データファイルの読込 --- (*1)
# data_file = "./ok-spam.pickle"
# save_file = "./ok-spam-model.pickle"
# data = pickle.load(open(data_file, "rb"))
# y = data[0] # ラベル
# x = data[1] # 単語の出現頻度

# # 100回、学習とテストを繰り返す --- (*2)
# count = 100
# rate = 0
# for i in range(count):
#     # データを学習用とテスト用に分割 --- (*3)
#     x_train, x_test, y_train, y_test = train_test_split(
#         x, y, test_size=0.2)
#     # 学習する --- (*4)
#     model = GaussianNB()
#     model.fit(x_train, y_train)
#     # 評価する ---(*5)
#     y_pred = model.predict(x_test)
#     acc = accuracy_score(y_test, y_pred)
#     # 評価結果が良ければモデルを保存 --- (*6)
#     if acc > 0.94: pickle.dump(model, open(save_file, "wb"))
#     print(acc)
#     rate += acc
# # 平均値を表示 --- (*7)
# print("----")
# print("average=", rate / count)

In [11]:
# import pickle
# import MeCab
# import numpy as np
# from sklearn.naive_bayes import GaussianNB

# # テストするテキスト --- (※1)
# test_text1 = """
# 会社から支給されているiPhoneの調子が悪いのです。
# 修理に出すので、しばらくはアプリのテストができません。
# """
# test_text2 = """
# 億万長者になる方法を教えます。
# すぐに以下のアドレスに返信して。
# """
# # ファイル名
# data_file = "./ok-spam.pickle"
# model_file = "./ok-spam-model.pickle"
# label_names = ['OK', 'SPAM']
# # 単語辞書を読み出す --- (※2)
# data = pickle.load(open(data_file, "rb"))
# word_dic = data[2]
# # MeCabの準備
# tagger = MeCab.Tagger()
# # 学習済みモデルを読み出す --- (※3) 
# model = pickle.load(open(model_file, "rb"))

# # テキストがスパムかどうか判定する --- (※4)
# def check_spam(text):
#     # テキストを単語IDのリストに変換し単語の頻出頻度を調べる
#     zw = np.zeros(word_dic['__id'])
#     count = 0
#     s = tagger.parse(text)
#     # 単語毎の回数を加算 --- (※5)
#     for line in s.split("\n"):
#         if line == "EOS": break
#         org =  line.split("\t")[3]# 単語の原型
#         if org in word_dic:
#             id = word_dic[org]
#             zw[id] += 1
#             count += 1
#     zw = zw / count #  --- (※6)
#     # 予測
#     pre = model.predict([zw])[0] #  --- (※7)
#     print("- 結果=", label_names[pre])

# check_spam(test_text1)
# check_spam(test_text2)

# TF-IDF

- 文章を数値ベクトルに変換。
- 単語の出現頻度に加え、文章全体における単語の重要度を考慮。
- 文書内おける特徴的な単語を見つけることを重視。
- 学習される全ての文書で、その単語がどのくらいの頻度で使われているか調べる。
    - 単語の出現回数を数えるだけでなく、出現頻度の高い単語のレートを下げ、特徴的な単語のレートを高く評価する方法で、単語をベクトル化。

In [12]:
# TF-IDFでテキストをベクトル化するモジュール
import MeCab
import pickle
import numpy as np

# MeCabの初期化 ---- (*1)
tagger = MeCab.Tagger(
    "-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd")
# グローバル変数 --- (*2)
word_dic = {'_id': 0} # 単語辞書
dt_dic = {} # 文書全体での単語の出現回数
files = [] # 全文書をIDで保存

def tokenize(text):
    '''MeCabで形態素解析を行う''' # --- (*3)
    result = []
    word_s = tagger.parse(text)
    for n in word_s.split("\n"):
        if n == 'EOS' or n == '': continue
        p = n.split("\t")[1].split(",")
        h, h2, org = (p[0], p[1], p[6])
        if not (h in ['名詞', '動詞', '形容詞']): continue
        if h == '名詞' and h2 == '数': continue
        result.append(org)
    return result

def words_to_ids(words, auto_add = True):
    '''単語一覧をIDの一覧に変換する''' # --- (*4)
    result = []
    for w in words:
        if w in word_dic:
            result.append(word_dic[w])
            continue
        elif auto_add:
            id = word_dic[w] = word_dic['_id']
            word_dic['_id'] += 1
            result.append(id)
    return result

def add_text(text):
    '''テキストをIDリストに変換して追加''' # --- (*5)
    ids = words_to_ids(tokenize(text))
    files.append(ids)

def add_file(path):
    '''テキストファイルを学習用に追加する''' # --- (*6)
    with open(path, "r", encoding="utf-8") as f:
        s = f.read()
        add_text(s)

def calc_files():
    '''追加したファイルを計算''' # --- (*7)
    global dt_dic
    result = []
    doc_count = len(files)
    dt_dic = {}
    # 単語の出現頻度を数える --- (*8)
    for words in files:
        used_word = {}
        data = np.zeros(word_dic['_id'])
        for id in words:
            data[id] += 1
            used_word[id] = 1
        # 単語tが使われていればdt_dicを加算 --- (*9)
        for id in used_word:
            if not(id in dt_dic): dt_dic[id] = 0
            dt_dic[id] += 1
        # 出現回数を割合に直す --- (*10)
        data = data / len(words) 
        result.append(data)
    # TF-IDFを計算 --- (*11)
    for i, doc in enumerate(result):
        for id, v in enumerate(doc):
            idf = np.log(doc_count / dt_dic[id]) + 1
            doc[id] = min([doc[id] * idf, 1.0])
        result[i] = doc
    return result

def save_dic(fname):
    '''辞書をファイルへ保存する''' # --- (*12)
    pickle.dump(
        [word_dic, dt_dic, files],
        open(fname, "wb"))

def load_dic(fname):
    '''辞書をファイルから読み込む''' # --- (*13)
    global word_dic, dt_dic, files
    n = pickle.load(open(fname, 'rb'))
    word_dic, dt_dic, files = n

def calc_text(text):
    ''' 辞書を更新せずにベクトル変換する ''' # --- (*14)
    data = np.zeros(word_dic['_id'])
    words = words_to_ids(tokenize(text), False)
    for w in words:
        data[w] += 1
    data = data / len(words)
    for id, v in enumerate(data):
        idf = np.log(len(files) / dt_dic[id]) + 1
        data[id] = min([data[id] * idf, 1.0])
    return data

# モジュールのテスト --- (*15)
if __name__ == '__main__':
    add_text('雨')
    add_text('今日は、雨が降った。')
    add_text('今日は暑い日だったけど雨が降った。')
    add_text('今日も雨だ。でも日曜だ。')
    print(calc_files())
    print(word_dic)

[array([1., 0., 0., 0., 0., 0.]), array([0.33333333, 0.42922736, 0.56438239, 0.        , 0.        ,
       0.        ]), array([0.2       , 0.25753641, 0.33862944, 0.47725887, 0.47725887,
       0.        ]), array([0.33333333, 0.42922736, 0.        , 0.        , 0.        ,
       0.79543145])]
{'_id': 6, '雨': 0, '今日': 1, '降る': 2, '暑い': 3, '日': 4, '日曜': 5}


# 文章をTF-IDFのデータベースに変換

```
pip install tfidf
```

In [13]:
import os, glob, pickle

# 変数の初期化
y = []
x = []

# ディレクトリ内のファイル一覧を処理 --- (*1)
def read_files(path, label):
    print("read_files=", path)
    files = glob.glob(path + "/*.txt")
    for f in files:
        if os.path.basename(f) == 'LICENSE.txt': continue
        add_file(f)
        y.append(label)

# ファイル一覧を読む --- (*2)
read_files('text/sports-watch', 0)
read_files('text/it-life-hack', 1)
read_files('text/movie-enter', 2)
read_files('text/dokujo-tsushin', 3)

# TF-IDFベクトルに変換 --- (*3)
x = calc_files()

# 保存 --- (*4)
pickle.dump([y, x], open('text/genre.pickle', 'wb'))
save_dic('text/genre-tdidf.dic')
print('ok')

read_files= text/sports-watch
read_files= text/it-life-hack
read_files= text/movie-enter
read_files= text/dokujo-tsushin
ok


# TF-IDFをNaiveBayesで学習

In [14]:
# import pickle
# from sklearn.naive_bayes import GaussianNB
# from sklearn.model_selection import train_test_split
# import sklearn.metrics as metrics
# import numpy as np

# # TF-IDFのデータベースを読み込む --- (*1)
# data = pickle.load(open("text/genre.pickle", "rb"))
# y = data[0] # ラベル
# x = data[1] # TF-IDF

# # 学習用とテスト用に分ける --- (*2)
# x_train, x_test, y_train, y_test = train_test_split(
#         x, y, test_size=0.2)

# # ナイーブベイズで学習 --- (*3)
# model = GaussianNB()
# model.fit(x_train, y_train)

# # 評価して結果を出力 --- (*4)
# y_pred = model.predict(x_test)
# acc = metrics.accuracy_score(y_test, y_pred)
# rep = metrics.classification_report(y_test, y_pred)

# print("正解率=", acc)
# print(rep)

# DeepLearningで学習

In [15]:
# import pickle
# from sklearn.model_selection import train_test_split
# import sklearn.metrics as metrics
# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from keras.optimizers import RMSprop
# import matplotlib.pyplot as plt
# import numpy as np
# import h5py

# # 分類するラベルの数 --- (*1)
# nb_classes = 4

# # データベースの読込 --- (*2)
# data = pickle.load(open("text/genre.pickle", "rb"))
# y = data[0] # ラベル
# x = data[1] # TF-IDF
# # ラベルデータをone-hotベクトルに直す --- (*3)
# y = keras.utils.np_utils.to_categorical(y, nb_classes)
# in_size = x[0].shape[0]

# # 学習用とテスト用を分ける --- (*4)
# x_train, x_test, y_train, y_test = train_test_split(
#         np.array(x), np.array(y), test_size=0.2)

# # MLPモデル構造を定義 --- (*5)
# model = Sequential()
# model.add(Dense(512, activation='relu', input_shape=(in_size,)))
# model.add(Dropout(0.2))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(nb_classes, activation='softmax'))

# # モデルをコンパイル --- (*6)
# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=RMSprop(),
#     metrics=['accuracy'])

# # 学習を実行 --- (*7)
# hist = model.fit(x_train, y_train,
#           batch_size=128, 
#           epochs=20,
#           verbose=1,
#           validation_data=(x_test, y_test))

# # 評価する ---(*8)
# score = model.evaluate(x_test, y_test, verbose=1)
# print("正解率=", score[1], 'loss=', score[0])

# # 重みデータを保存 --- (*9)
# model.save_weights('./text/genre-model.hdf5')

# # 学習の様子をグラフへ描画 --- (*10)
# plt.plot(hist.history['accuracy'])
# plt.plot(hist.history['val_accuracy'])
# plt.title('Accuracy')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

# 自分で文章を指定して判定

In [16]:
# import pickle
# import numpy as np
# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from keras.optimizers import RMSprop
# from keras.models import model_from_json

# # 独自のテキストを指定 --- (*1)
# text1 = """
# 野球を観るのは楽しいものです。
# 試合だけでなくインタビューも楽しみです。
# """
# text2 = """
# 常にiPhoneとiPadを持っているので、
# 二口あるモバイルバッテリがあると便利。
# """
# text3 = """
# 幸せな結婚の秘訣は何でしょうか。
# 夫には敬意を、妻には愛情を示すことが大切。
# """

# # TF-IDFの辞書を読み込む --- (*2)
# load_dic("text/genre-tdidf.dic")

# # Kerasのモデルを定義して重みデータを読み込む --- (*3)
# nb_classes = 4
# dt_count = len(dt_dic)
# model = Sequential()
# model.add(Dense(512, activation='relu', input_shape=(dt_count,)))
# model.add(Dropout(0.2))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(nb_classes, activation='softmax'))
# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=RMSprop(),
#     metrics=['accuracy'])
# model.load_weights('./text/genre-model.hdf5')

# # テキストを指定して判定 --- (*4)
# def check_genre(text):
#     # ラベルの定義
#     LABELS = ["スポーツ", "IT", "映画", "ライフ"]
#     # TF-IDFのベクトルに変換 -- (*5)
#     data = calc_text(text)
#     # MLPで予測 --- (*6)
#     pre = model.predict(np.array([data]))[0]
#     n = pre.argmax()
#     print(LABELS[n], "(", pre[n], ")")
#     return LABELS[n], float(pre[n]), int(n) 

# if __name__ == '__main__':
#     check_genre(text1)
#     check_genre(text2)
#     check_genre(text3)

# Webで使える文章ジャンル判定アプリ

In [17]:
# import json
# import flask
# from flask import request

# # テキストを指定して判定 --- (*4)
# def check_genre(text):
#     # ラベルの定義
#     LABELS = ["スポーツ", "IT", "映画", "ライフ"]
#     # TF-IDFのベクトルに変換 -- (*5)
#     data = calc_text(text)
#     # MLPで予測 --- (*6)
#     pre = model.predict(np.array([data]))[0]
#     n = pre.argmax()
#     print(LABELS[n], "(", pre[n], ")")
#     return LABELS[n], float(pre[n]), int(n) 


# # ポート番号 --- (*1)
# TM_PORT_NO = 8888
# # HTTPサーバを起動
# app = flask.Flask(__name__)

# # 一度、ジャンル判定のテストをする
# label, per, no = check_genre("テスト")
# print("> テスト --- ", label, per, no)

# # ルートへアクセスした場合 --- (*2)
# @app.route('/', methods=['GET'])
# def index():
#     with open("index.html", "rb") as f:
#         return f.read()

# # /api へアクセスした場合
# @app.route('/api', methods=['GET'])
# def api():
#     # URLパラメータを取得 --- (*3)
#     q = request.args.get('q', '')
#     if q == '':
#       return '{"label": "空です", "per":0}'
#     print("q=", q)
#     # テキストのジャンル判定を行う --- (*4)
#     label, per, no = check_genre(q)
#     # 結果をJSONで出力
#     return json.dumps({
#       "label": label, 
#       "per": per,
#       "genre_no": no
#     })
    
# if __name__ == '__main__':
#     # サーバを起動
#     app.run(debug=True, host='0.0.0.0', port=TM_PORT_NO)